In [1]:
#Importing all the required libraries

import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn import svm

import pandas as pd
from io import StringIO
import io
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np
from sklearn.preprocessing import LabelEncoder

from keras.layers.recurrent import LSTM, GRU
from keras.models import Sequential
from keras import optimizers

from keras.layers.core import Dense, Activation, Dropout

We have cleaned the data by dropping unnecessary values as well as filling mean values in place of Null values.

In [2]:
finance=pd.read_csv("./targetfirm_prediction_dataset_small.csv")
finance.drop(['ca','Unnamed: 0'],axis=1,inplace=True)
perc=50.0
min_count = int(((100-perc)/100)*finance.shape[1]+1)
finance=finance.dropna(axis=0,thresh=min_count)
for i in finance.columns[finance.isnull().any(axis=0)]:     #---Applying Only on variables with NaN values
    finance[i].fillna(finance[i].mean(),inplace=True)

LSTM

In [4]:
finance_lstm  = finance[['target', 'gvkey', 'at','revt','ib','capx','ceq','che','emp','invt','re','xrd','dltt',
                        'dv','sic']].copy()

In [6]:
X_columns=[x for x in finance_lstm.columns if x!='target']

In [7]:
X_columns

['gvkey',
 'at',
 'revt',
 'ib',
 'capx',
 'ceq',
 'che',
 'emp',
 'invt',
 're',
 'xrd',
 'dltt',
 'dv',
 'sic']

We have defined the model type as sequential and added LSTM here. We have added accuracy metrics to ensure model is working well and applied groupby on 'gvkey'.

In [8]:
features1=len(X_columns)
model = Sequential()
model.add(LSTM(100, activation='tanh', return_sequences=True, input_shape=(1, features1)))
model.add(LSTM(49, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer="rmsprop", loss='binary_crossentropy', metrics=['accuracy','AUC','Precision','Recall'])


grouped=finance_lstm.groupby('gvkey')

We have applied a for loop to run the lstm model and dropped all the unnecessary values. 

In [9]:
for group in grouped:
    df_batch=pd.DataFrame(columns=['gvkey','at','revt','ib','capx','ceq','che','emp','invt','re','xrd','dltt','dv','sic','target'])
    for gvkey in group:
        filter=finance_lstm['gvkey']==gvkey 
        for key,item in finance_lstm[filter].dropna().iterrows():
            df_batch=df_batch.append({'gvkey':item['gvkey'],'at':item['at'],'revt':item['revt'],'ib':item['ib'],'capx':item['capx'],'ceq':item['ceq'],'che':item['che'],'emp':item['emp'],'invt':item['invt'],'re':item['re'],'xrd':item['xrd'],'dltt':item['dltt'],'dv':item['dv'],'sic':item['sic'],'target':item['target']},ignore_index=True)
    #print("\n",df_batch)
    X=df_batch[X_columns]
    X = np.resize(X,(X.shape[0],1,X.shape[1]))
    y=df_batch['target']
    print("\n",X)
    model.fit(X,y,batch_size=len(X), 
          epochs=10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  after removing the cwd from sys.path.



 [[[ 1.00400000e+03  7.01854000e+02  8.74255000e+02  1.85310000e+01
    1.31340000e+01  3.40212000e+02  1.38090000e+01  2.50000000e+00
    3.20590000e+02  2.04065000e+02  1.06459562e+02  1.79987000e+02
    9.15700000e+00  5.08000000e+03]]

 [[ 1.00400000e+03  7.10199000e+02  6.38721000e+02 -5.89390000e+01
    1.21120000e+01  3.10235000e+02  3.45220000e+01  2.20000000e+00
    2.86588000e+02  1.39603000e+02  1.06459562e+02  2.17699000e+02
    4.43000000e+00  5.08000000e+03]]

 [[ 1.00400000e+03  6.86621000e+02  6.06337000e+02 -1.24100000e+01
    9.93000000e+00  2.94988000e+02  2.91540000e+01  2.10000000e+00
    2.59954000e+02  1.24106000e+02  1.06459562e+02  1.64658000e+02
    7.97000000e-01  5.08000000e+03]]

 [[ 1.00400000e+03  7.09292000e+02  6.51958000e+02  3.50400000e+00
    1.02860000e+01  3.01684000e+02  4.10100000e+01  2.30000000e+00
    2.47245000e+02  1.31884000e+02  1.06459562e+02  2.48666000e+02
    0.00000000e+00  5.08000000e+03]]

 [[ 1.00400000e+03  7.32230000e+02  7.4784

Now we follow the same steps for GRU here as we didi for LSTM.

In [10]:
features2=len(X_columns)
model = Sequential()
model.add(GRU(100, activation='tanh', return_sequences=True, input_shape=(1, features2)))
model.add(GRU(49, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer="rmsprop", loss='binary_crossentropy', metrics=['accuracy','AUC','Precision','Recall'])


grouped2=finance_lstm.groupby('gvkey')

In [11]:
for group in grouped2:
    df_batch=pd.DataFrame(columns=['gvkey','at','revt','ib','capx','ceq','che','emp','invt','re','xrd','dltt','dv','sic','target'])
    for gvkey in group:
        filter=finance_lstm['gvkey']==gvkey 
        for key,item in finance_lstm[filter].dropna().iterrows():
            df_batch=df_batch.append({'gvkey':item['gvkey'],'at':item['at'],'revt':item['revt'],'ib':item['ib'],'capx':item['capx'],'ceq':item['ceq'],'che':item['che'],'emp':item['emp'],'invt':item['invt'],'re':item['re'],'xrd':item['xrd'],'dltt':item['dltt'],'dv':item['dv'],'sic':item['sic'],'target':item['target']},ignore_index=True)
    #print("\n",df_batch)
    X=df_batch[X_columns]
    X = np.resize(X,(X.shape[0],1,X.shape[1]))
    y=df_batch['target']
    print("\n",X)
    model.fit(X,y,batch_size=len(X), 
          epochs=10)


 [[[ 1.00400000e+03  7.01854000e+02  8.74255000e+02  1.85310000e+01
    1.31340000e+01  3.40212000e+02  1.38090000e+01  2.50000000e+00
    3.20590000e+02  2.04065000e+02  1.06459562e+02  1.79987000e+02
    9.15700000e+00  5.08000000e+03]]

 [[ 1.00400000e+03  7.10199000e+02  6.38721000e+02 -5.89390000e+01
    1.21120000e+01  3.10235000e+02  3.45220000e+01  2.20000000e+00
    2.86588000e+02  1.39603000e+02  1.06459562e+02  2.17699000e+02
    4.43000000e+00  5.08000000e+03]]

 [[ 1.00400000e+03  6.86621000e+02  6.06337000e+02 -1.24100000e+01
    9.93000000e+00  2.94988000e+02  2.91540000e+01  2.10000000e+00
    2.59954000e+02  1.24106000e+02  1.06459562e+02  1.64658000e+02
    7.97000000e-01  5.08000000e+03]]

 [[ 1.00400000e+03  7.09292000e+02  6.51958000e+02  3.50400000e+00
    1.02860000e+01  3.01684000e+02  4.10100000e+01  2.30000000e+00
    2.47245000e+02  1.31884000e+02  1.06459562e+02  2.48666000e+02
    0.00000000e+00  5.08000000e+03]]

 [[ 1.00400000e+03  7.32230000e+02  7.4784

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  after removing the cwd from sys.path.


1/1 [==============================] - 4s 4s/step - loss: 0.7035 - accuracy: 0.3333 - auc: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 15ms/step - loss: 0.4353 - accuracy: 1.0000 - auc: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 15ms/step - loss: 0.3177 - accuracy: 1.0000 - auc: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 12ms/step - loss: 0.2491 - accuracy: 1.0000 - auc: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 12ms/step - loss: 0.2032 - accuracy: 1.0000 - auc: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 12ms/step - loss: 0.1702 - accuracy: 1.0000 - auc: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 